In [1]:
from numpy import array
import keras
from keras.callbacks import CSVLogger 
import numpy as np
import matplotlib.pyplot as plt
from pickle import dump
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Input, Bidirectional
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# load doc into memory
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

# load
in_filename = r'../input/5gram-tokenizer/5gram_token.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [3]:
len(lines)

In [4]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [5]:
vocab_size

In [6]:
len(sequences)

In [7]:
length = 6

In [8]:
sequences = array(sequences)
sequences

In [9]:
c=1
h=0
while(c!=0):
    limit = sequences.size
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)     

In [10]:
print(sequences[564])
print(sequences.size)

In [11]:
sequences = np.array(sequences.tolist())
sequences 

In [12]:
sequences.shape

In [13]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [14]:
class My_Callback(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        def on_epoch_end(self, epoch, logs=None):
          logs = logs or {}
        for k in self.params['metrics']:
            if k in logs:
                print("Name: %s, Value: %s" % (k, logs[k]))
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        return

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint
# define model
model = Sequential()
model.add(Input(shape = (X.shape[1],)))
model.add(Embedding(input_dim = vocab_size, output_dim = 1000, input_length = X.shape[1]))
model.add(Bidirectional(LSTM(1000, activation="relu", return_sequences=True)))
model.add(Bidirectional(LSTM(1000)))
model.add(Dropout(0.6))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
print(model.summary())
# compile model
checkpoint = ModelCheckpoint("./5gram_bilstm.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001),metrics='accuracy')

In [16]:
# fit model
history = model.fit(X, y, epochs=50, batch_size=128, callbacks=[checkpoint])

In [19]:
score, acc = model.evaluate(X, y, batch_size = 128) 
   
print('Test score:', score) 
print('Test accuracy:', acc)

In [17]:
from matplotlib import pyplot as plt
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
epochs = range(1, len(loss) +1)
plt.plot(epochs,loss,'y',label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [18]:
from matplotlib import pyplot as plt
#plot the training and validation accuracy and loss at each epoch
accuracy = history.history['accuracy']
epochs = range(1, len(accuracy) +1)
plt.plot(epochs,accuracy,'y',label='Training accuracy')
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [20]:
# save the tokenizer
dump(tokenizer, open(r'token_5gram_bilstm.pkl', 'wb'))

In [21]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('./5gram_bilstm.h5')
tokenizer = pickle.load(open('./token_5gram_bilstm.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [22]:
while(True):
  text = input("এখানে টাইপ করুন: ")
  
  if text == "সমাপ্তি":
      print("প্রোগ্রাম সমাপ্ত....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue